In [1]:
import yaml
import os
import pymupdf
from os.path import join as pjoin

In [2]:
root = '/home/quamer23nasim38/Superhero-Character-Based-On-RAG-AI-Using-Haystack-And-Qdrant'
data_folder = 'data'
config_file = 'config.yaml'

In [3]:
with open(pjoin(root, data_folder, config_file), 'r') as f:
    config = yaml.safe_load(f)

In [1]:
import torch
torch.cuda.is_available()

False

In [4]:
all_movie_scripts = os.listdir(pjoin(root, data_folder))

In [5]:
def extract_text_from_pdf(pdf_path):
    pdf = pymupdf.open(pdf_path)
    text = ''
    for page in pdf:
        text += page.get_text()
    return text

In [6]:
for superhero in config['LIST_OF_SUPERHEROES']:
    superhero_script = []
    for script in all_movie_scripts:
        script_path = pjoin(root, data_folder, script)
        script_text = extract_text_from_pdf(script_path)
        break
    break

In [7]:
def extract_spiderman_dialogues(text):
    # Remove screen explanations
    lines = text.split('\n')
    dialogues = [line.strip() for line in lines if line.isupper() or (line and not line.startswith('(') and not re.search(r'[a-z]', line))]
    
    # Find all dialogues
    spiderman_dialogues = []
    for i, line in enumerate(dialogues):
        if 'SPIDER-MAN' in line:
            # Collect two preceding dialogues as context
            context = dialogues[max(0, i-2):i+1]
            spiderman_dialogues.append('\n'.join(context))
    
    return spiderman_dialogues

In [8]:
import re
spiderman_dialogues = extract_spiderman_dialogues(script_text)

In [16]:
import re

text = """Some text before Spider-Man
Some text with Spider-Man and more text
A line without keywords
And some more text with Peter
More text without keywords
More text with Spider-Man again."""

# Split the text into lines
lines = text.split('\n')

# Initialize a list to hold the results
result = []
current_text = []

# Process each line
for line in lines:
    if "Spider-Man" in line or "Peter" in line:
        if current_text:
            result.append('\n'.join(current_text))
            current_text = []
    current_text.append(line)

# Append the last collected text
if current_text:
    result.append('\n'.join(current_text))

# Join the result with new lines for clarity
final_result = '\n\n'.join(result)

print(result)


['Some text before Spider-Man', 'Some text with Spider-Man and more text\nA line without keywords', 'And some more text with Peter\nMore text without keywords', 'More text with Spider-Man again.']


In [12]:
script_text.split('SPIDER-MAN')

['Written as\nWritten by\nChris McKenna & Erik Sommers\n',
 ": NO WAY HOME\nSERENITY NOW\n                                                                1.\nOVER SONY LOGO: \nNY1 REPORTER (V.O.)\nWe come to you now with revelations \nabout last week’s attack in London.  \nAn anonymous source provided this \nvideo.  It shows Quentin Beck, aka \nMysterio, moments before his death.  \nWarning: you may find this video \ndisturbing.  \nOVER COLUMBIA LOGO:\nQUENTIN BECK (V.O.)\nI managed to send the Elemental \nback through the dimensional rift, \nbut I don't think I'm gonna make it \noff this bridge alive.  Spider-Man \nattacked me for some reason.  He \nhas an army of weaponized drones, \nStark technology.  He's saying he's \nthe only one who's gonna be the new \nIron Man, no one else.  \nE.D.I.T.H. (V.O.)\nAre you sure you want to commence \nthe drone attack?  There will be \nsignificant casualties.\n",
 " (V.O.)\nDo it!  Execute them all!\nDRONES FIRE!  EXPLOSIONS from the Tower Bridge.

In [11]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"


pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a movie dialogue separator. From the context you are given, separate the dialogues and provide the dialogues of Spiderman."},
    {"role": "user", "content": "Give me the dialogues of Spiderman from the following text:\n\n" + script_text},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]



outputs = pipeline(
    messages,
    max_new_tokens=4096,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)



/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
print(outputs[0]["generated_text"][-1]['content'])

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-70B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a movie dialogue separator. From the context you are given, separate the dialogues and provide the dialogues of Spiderman."},
    {"role": "user", "content": "Give me the dialogues of Spiderman from the following text:\n\n" + script_text[:4000]},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])


In [ ]:
!pip install PyPDF2==2.9

In [ ]:
import PyPDF2
PyPDF2.__version__

In [ ]:

# Open the PDF file
file_path = "/home/quamer23nasim38/Superhero-Character-Based-On-RAG-AI-Using-Haystack-And-Qdrant/data/spider-man-no-way-home-2021.pdf"
pdf_file = open(file_path, "rb")

# Create a PDF reader object
pdf_reader = PyPDF2.PdfFileReader(pdf_file)

# Get the total number of pages
total_pages = pdf_reader.numPages

# Function to extract text from each page and filter Spider-Man's dialogues
def extract_spiderman_dialogues(reader, total_pages):
    dialogues = []
    spider_man_lines = False
    for page_num in range(total_pages):
        page = reader.getPage(page_num)
        text = page.extract_text()
        lines = text.split("\n")
        
        for line in lines:
            if line.strip() == "SPIDER-MAN":
                spider_man_lines = True
                dialogues.append(line.strip())
            elif spider_man_lines and line.strip() and not line.startswith(" "):
                dialogues.append(line.strip())
            elif not line.strip():
                spider_man_lines = False
    
    return dialogues

# Extract Spider-Man's dialogues
spiderman_dialogues = extract_spiderman_dialogues(pdf_reader, total_pages)

# Close the PDF file
pdf_file.close()

spiderman_dialogues


In [ ]:
spiderman_dialogues